In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
import warnings
warnings.filterwarnings(action='ignore')

In [7]:
import os
os.chdir('/content/drive/MyDrive/Projects/reddit-vote-predictor')
import sys
import pandas as pd
from datetime import datetime
# Import custom module
from scripts.RedditScrape import * # imports a class `RedditScrape`

In [8]:
# Reddit's own api:
!pip3 install psaw
!pip3 install praw
import psaw
import praw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 188 kB 38.0 MB/s 
     |████████████████████████████████| 55 kB 3.9 MB/s 


In [ ]:
# Set date range from which to extract data for two subreddits
# NOTE: ater running the data scraping and cleaning pipeline, one year of data
# resulted in around 1000 images, so we'll add time to the date range
# 10 years from the start of 2011 to the end of 2020
start_ = datetime(2011, 1, 1)
end_ = datetime(2020, 12, 31)
# Run the scraping pipeline using custom RedditScrape class imported above
cat = RedditScrape('cat', start_, end_)
cat.scrape_posts()
cat.posts.to_csv('data/tbl/cat_posts.csv', index=False)
dog = RedditScrape('dog', start_, end_)
dog.scrape_posts()
dog.posts.to_csv('data/tbl/dog_posts.csv', index=False)

In [15]:
# If the above is not run, read in the data
cat = pd.read_csv('data/tbl/cat_posts.csv', index_col = None)
dog = pd.read_csv('data/tbl/dog_posts.csv', index_col = None)

## Using `PRAW`
Now use `praw` to fetch additional information about each post now that it has been efficiently scraped using `psaw`. This will help us gather more information that will help more accurately filter for good posts/images, and acquire information that might be useful in modeling `score`.

Use the `id` field of the `psaw` results to get more information for each post.

In [ ]:
# reddit = praw.Reddit(
#     client_id = 'client id',
#     client_secret = 'client secret',
#     username = 'username',
#     password = 'password',
#     user_agent = 'my user agent'
# )
# Fill in all fields above
# Docs: https://praw.readthedocs.io/en/stable/getting_started/quick_start.html
# Now simply lookup the scraped posts on reddit's praw using the id obtained by psaw
# Then use the following method to get more data for each post:
# post = reddit.submission(id = ID)
# where ID is the value in column "id" in the data obtained from psaw

The above arguments are saved in a secret json file.

In [9]:
import json
with open('reddit_args.json') as f:
  args = json.load(f)

### Now use the args (which is a dict of keyword arguments) to set up the instance of the reddit praw api Reddit class

In [10]:
reddit = praw.Reddit(**args, check_for_async=False)

Using each post in the data scraped using `psaw`, we search for the exact submission using the `reddit` object created above. Each submission has attributes:

* `submission.score` # the score upvotes - downvotes
* `submission.ups` # the number of upvotes
* `submission.downs` # the number of downvotes

Note that `score` also downloaded with he `psaw` scrape but is not as accurate as the one downloaded from `praw` and we'll use that instead.


In [ ]:
# Set up a function to get the desired data if the submission is still up.
# If it no longer exists then it would throw a NotFound error, so use the
# class method fetch_data() to try to pull the data initially, and if it exists
# actually obtain the 3 columns desired
def try_get_data(submission):
  try:
    submission._fetch_data()
  except:
    out = [None]*3
  else:
    out = [getattr(submission, attr) for attr in ['score', 'ups', 'downs']]
  return out

In [ ]:
# Set up lists to store outputs
cat_praw = []
dog_praw = []
# Iterate through submissions in cat and dog data
for i in range(cat.shape[0]):
  id_ = cat.id[i] # get the submission's id
  submission = reddit.submission(id_) # use to PRAW to find this submission
  data_ = try_get_data(submission) # try to get score, upvotes, and downvotes if exists
  cat_praw.append(data_) # add to list of outputs
# convert output list to data frame
cat_scores = pd.DataFrame(cat_praw, columns = ['praw_score', 'upvotes', 'downvotes'])

# Repeat for dog
for i in range(dog.shape[0]):
  id_ = dog.id[i]
  submission = reddit.submission(id_)
  data_ = try_get_data(submission)
  dog_praw.append(data_)
dog_scores = pd.DataFrame(dog_praw, columns = ['praw_score', 'upvotes', 'downvotes'])

### Now combine the variables scraped from `praw` with data from `psaw`

In [ ]:
cat_new = pd.concat([cat, cat_praw], axis = 1)
dog_new = pd.concat([dog, dog_praw], axis= 1)
cat_new.to_csv('data/tbl/dog_posts.csv', index=False)
dog_new.to_csv('data/tbl/dog_posts.csv', index=False)